In [1]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.fft import rfft, rfftfreq
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import lightgbm as lgb

In [2]:
COLUMNS = ["timestamp","ax","ay","az","gx","gy","gz","temp"]

def load_csv(path, label):
    df = pd.read_csv(path, names=COLUMNS)
    df["label"] = label
    return df

idle_df    = load_csv("idle.csv",    0)
walking_df = load_csv("walking.csv", 1)
running_df = load_csv("running.csv", 2)

df = pd.concat([idle_df, walking_df, running_df], ignore_index=True)


In [3]:
WINDOW_SIZE = 200
STEP_SIZE   = 50

def create_windows(df):
    windows = []
    labels  = []

    for label in df["label"].unique():
        temp = df[df["label"] == label]

        for i in range(0, len(temp) - WINDOW_SIZE, STEP_SIZE):
            window = temp.iloc[i:i+WINDOW_SIZE]
            windows.append(window)
            labels.append(label)

    return windows, np.array(labels)


In [4]:
def extract_features(window):
    features = {}

    for col in ["ax","ay","az","gx","gy","gz"]:
        x = window[col].values

        features[f"{col}_mean"] = np.mean(x)
        features[f"{col}_std"]  = np.std(x)
        features[f"{col}_min"]  = np.min(x)
        features[f"{col}_max"]  = np.max(x)
        features[f"{col}_skew"] = skew(x)
        features[f"{col}_kurt"] = kurtosis(x)
        features[f"{col}_energy"] = np.sum(x**2) / len(x)

    # Acceleration magnitude (VERY IMPORTANT)
    acc_mag = np.sqrt(
        window["ax"]**2 +
        window["ay"]**2 +
        window["az"]**2
    )

    features["acc_mag_mean"] = acc_mag.mean()
    features["acc_mag_std"]  = acc_mag.std()
    features["acc_mag_energy"] = np.sum(acc_mag**2) / len(acc_mag)

    # Dominant frequency (FFT)
    fft_vals = np.abs(rfft(acc_mag))
    freqs = rfftfreq(len(acc_mag), d=1/50)
    features["dom_freq"] = freqs[np.argmax(fft_vals)]

    # Temperature
    features["temp_mean"] = window["temp"].mean()
    features["temp_std"]  = window["temp"].std()

    return features


In [5]:
df

,timestamp,ax,ay,az,gx,gy,gz,temp,label
0,1791011,-3607,-4712,208,6554,-1013,-1693,44.06,0
1,1791031,-4335,-4960,676,7664,-984,-2271,43.97,0
2,1791051,-5411,-5180,-84,9600,-1102,-3164,44.01,0
3,1791071,-5399,-5184,-268,12813,-1045,-4171,43.54,0
4,1791091,-5107,-4488,-112,15192,-1169,-5705,44.06,0
...,...,...,...,...,...,...,...,...,...
37536,1483571,-7311,-3552,-7428,2567,-3565,3683,40.72,2
37537,1483591,-5899,-4864,-7736,2198,-3179,5284,40.72,2
37538,1483611,-4075,-5832,-7836,2153,-3143,6447,40.72,2
37539,1483631,-2471,-6156,-7728,2147,-3485,7074,40.67,2


In [6]:
df["acc_mag"] = np.sqrt(
        df["ax"]**2 +
        df["ay"]**2 +
        df["az"]**2
    )

In [7]:
df["gyro_mag"] = np.sqrt(
        df["gx"]**2 +
        df["gy"]**2 +
        df["gz"]**2
    )

In [8]:
df

,timestamp,ax,ay,az,gx,gy,gz,temp,label,acc_mag,gyro_mag
0,1791011,-3607,-4712,208,6554,-1013,-1693,44.06,0,5937.731638,6844.511232
1,1791031,-4335,-4960,676,7664,-984,-2271,43.97,0,6621.993733,8053.731620
2,1791051,-5411,-5180,-84,9600,-1102,-3164,44.01,0,7491.219994,10167.856215
3,1791071,-5399,-5184,-268,12813,-1045,-4171,43.54,0,7489.651594,13515.259339
4,1791091,-5107,-4488,-112,15192,-1169,-5705,44.06,0,6799.715950,16269.924708
...,...,...,...,...,...,...,...,...,...,...,...
37536,1483571,-7311,-3552,-7428,2567,-3565,3683,40.72,2,11011.022160,5732.643631
37537,1483591,-5899,-4864,-7736,2198,-3179,5284,40.72,2,10876.690351,6546.594611
37538,1483611,-4075,-5832,-7836,2153,-3143,6447,40.72,2,10583.985308,7488.502320
37539,1483631,-2471,-6156,-7728,2147,-3485,7074,40.67,2,10184.505928,8172.900954


In [9]:
corrmatrix = df.corr()
corrmatrix["label"].sort_values(ascending=False)

label        1.000000
gyro_mag     0.756660
acc_mag      0.556906
az           0.415433
ay           0.258811
gz           0.178350
ax           0.077621
gx          -0.027629
gy          -0.107074
timestamp   -0.547471
temp        -0.771407
Name: label, dtype: float64

In [11]:
idle_walking =df[df["label"]==0]

In [18]:
idle_walking.describe()

,timestamp,ax,ay,az,gx,gy,gz,temp,label,acc_mag,gyro_mag
count,1.275900e+04,12759.000000,12759.000000,12759.000000,12759.000000,12759.000000,12759.000000,12759.000000,12759.0,12759.000000,12759.000000
mean,1.918591e+06,-3323.494710,692.052355,-5873.925856,1072.908457,679.922016,-426.606004,44.112053,0.0,7407.720112,2499.890102
std,7.366701e+04,675.388226,2938.485056,1155.110195,3043.113952,1159.852337,1285.071591,0.175020,0.0,1256.053767,2793.354348
min,1.791011e+06,-9987.000000,-11700.000000,-12712.000000,-32048.000000,-8188.000000,-11274.000000,43.490000,0.0,2172.631814,103.990384
25%,1.854801e+06,-3667.000000,-1268.000000,-6692.000000,565.000000,426.000000,-628.000000,44.010000,0.0,6610.896838,1237.586159
50%,1.918591e+06,-3367.000000,1844.000000,-5756.000000,1082.000000,651.000000,-382.000000,44.060000,0.0,7236.493695,1563.921354
75%,1.982381e+06,-2983.000000,2888.000000,-5204.000000,1590.000000,861.500000,-178.000000,44.200000,0.0,8285.789803,2503.834154
max,2.046171e+06,1945.000000,10528.000000,1192.000000,28834.000000,16617.000000,9459.000000,44.810000,0.0,14262.686738,32302.549652


In [20]:
walking_walking =df[df["label"]==1]
walking_walking.describe()

,timestamp,ax,ay,az,gx,gy,gz,temp,label,acc_mag,gyro_mag
count,1.242700e+04,12427.000000,12427.000000,12427.000000,12427.000000,12427.000000,12427.000000,12427.000000,12427.0,12427.000000,12427.000000
mean,9.029510e+05,-3813.241249,-1678.194898,-6155.794641,1090.928543,582.347791,-398.331697,43.212018,1.0,7864.211270,8332.149301
std,7.175020e+04,1655.782090,2216.767837,1897.011013,6089.299714,4140.178064,5777.060573,0.616440,0.0,2158.455402,4455.651833
min,7.786910e+05,-11199.000000,-13432.000000,-13916.000000,-31652.000000,-14351.000000,-16590.000000,42.130000,1.0,2587.279846,192.942997
25%,8.408210e+05,-4955.000000,-2902.000000,-7232.000000,-3013.500000,-2216.000000,-4621.000000,42.740000,1.0,6293.232906,4734.931626
50%,9.029510e+05,-3687.000000,-1284.000000,-5988.000000,1394.000000,847.000000,-494.000000,42.980000,1.0,7689.156586,7999.459294
75%,9.650810e+05,-2615.000000,-236.000000,-4752.000000,5343.000000,3537.000000,3815.500000,43.540000,1.0,9117.422481,11361.699215
max,1.027211e+06,1421.000000,10504.000000,-184.000000,32271.000000,11835.000000,15506.000000,45.000000,1.0,16602.307340,34256.229054


In [21]:
running_walking =df[df["label"]==2]
running_walking.describe()

,timestamp,ax,ay,az,gx,gy,gz,temp,label,acc_mag,gyro_mag
count,1.235500e+04,12355.000000,12355.000000,12355.000000,12355.000000,12355.000000,12355.000000,12355.000000,12355.0,12355.000000,12355.000000
mean,1.360111e+06,-2188.243545,4316.639660,129.365034,515.897208,-1780.285552,4654.066289,41.908370,2.0,14756.759969,25196.286050
std,7.133451e+04,10095.719408,8067.373902,8632.701082,12534.357799,15591.455051,18855.253180,1.111131,0.0,6869.529004,12079.647361
min,1.236571e+06,-20843.000000,-19228.000000,-25208.000000,-32687.000000,-32767.000000,-32755.000000,40.250000,2.0,404.565199,639.965624
25%,1.298341e+06,-9613.000000,-1894.000000,-6748.000000,-7237.000000,-13319.000000,-10038.000000,40.910000,2.0,9661.873978,15439.899787
50%,1.360111e+06,-1207.000000,3784.000000,-380.000000,1140.000000,-2353.000000,5019.000000,41.710000,2.0,14000.401459,24374.145216
75%,1.421881e+06,5921.000000,9834.000000,5950.000000,8939.000000,10566.000000,20319.500000,42.600000,2.0,19066.077977,34999.841049
max,1.483651e+06,24389.000000,32107.000000,22639.000000,32712.000000,32764.000000,32762.000000,44.670000,2.0,44141.589222,56295.014291
